# Lakehouse - Spark and delta lake - ecommerce example 
### We will build a lakehouse architecture utilising delta lakes, an end to end ETL pipeline in Fabric/Synapse. We will show spark structured streaming, delta lake's change data feed and checkpoint folders. 

### The use case here I am taking is of a Commerce company that has an ecommerce website as well as traditional retail stores. They want to analyse the online clickstream data to better understand their customers. 

### Clickstream data is data about how users interact with your ecommerce websites, what ads they click, what products they view, which pages they spend most time on. Behavioural data that can give you insights into your products and customers so you can better market to your customer base. Its important to start with the vision of any of these data projects. In my case, it could be to eventually develop ML models to provide product recommendations to my customers or to understand whether customers do not like any particular products, understand the churn rate.

### We will use Dynamics products and customers data in data lake to do lookups and joins to enrich this raw data or bronze delta table and create more refined tables, or silver delta table. Finally do some aggregation and create a Gold delta table.

### A good tutorial: https://learn.microsoft.com/en-us/training/modules/work-delta-lake-tables-fabric/

In [1]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.functions  import from_unixtime
from pyspark.sql.functions  import to_date
from pyspark.sql import Row
from pyspark.sql.functions import to_json, struct
from pyspark.sql import functions as F
import random
import time

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 3, Finished, Available)

In [2]:
#source
lakehouse_workspace = "yourlakehouseworkspace"

lakedatabase = "dataverse_xxxxx"

source_table1 = "custtable"
source_table2 = "ecoresproduct"

lake_source_table1 = "dataverse_xxxxx.custtable"
lake_source_table2 = "dataverse_xxxxx.ecoresproduct"

path_to_source = f'abfss://{lakehouse_workspace}@onelake.dfs.fabric.microsoft.com/{lakedatabase}.Lakehouse/Tables'

path_to_source_table1 = path_to_source + "/" + source_table1
path_to_source_table2 = path_to_source + "/" + source_table2

#target
lakehouse = "yourtargetlakehouse" # target lakehouse - create it first if does not exist and add it to notebook

#define storage to read historical data 
datalakename = "yourdatalakename"
container = "yourcontainer"
path_to_storage = f'abfss://{container}@{datalakename}.dfs.core.windows.net'


StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 4, Finished, Available)

In [3]:
# Reading customer data files in a dataframe

df_cust= spark.read.format("delta").load(path_to_source_table1)

display(df_cust.limit(10))

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 5, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5a9a4ff6-acb8-48de-ac13-5e17bda4e3ec)

In [4]:
# rename columns that we need and create a new dataframe
df_custSmall =  df_cust.selectExpr(
    'Id as Id',
    'accountnum AS CustomerId',
    'custgroup AS CustomerGroup',
    'currency AS Currency',
    'IsDelete AS IsDelete')

display(df_custSmall.limit(10))

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 473615a6-2d17-4452-8333-5c50dbb221a7)

In [5]:
#create a view from dataframe
df_custSmall.createOrReplaceTempView("vw_Customers")


StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 7, Finished, Available)

In [6]:
# Reading product files in a dataframe
df_product = spark.read.format("delta").load(path_to_source_table2)

display(df_product.limit(10))

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, 746c52f2-4c58-44f2-9073-704cd2304740)

In [7]:
# rename columns that we need and create a new dataframe
df_productSmall =  df_product.selectExpr(
    'displayproductnumber AS ProductId',
    'searchname AS ProductName',
    'IsDelete AS IsDelete')

display(df_productSmall.limit(10))

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 9, Finished, Available)

SynapseWidget(Synapse.DataFrame, 59f249cf-1a66-4784-bf4d-9b891ea6887a)

In [8]:
#create a view from dataframe
df_productSmall.createOrReplaceTempView("vw_Products")


StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 10, Finished, Available)

## Define schema of incoming clickstream data

In [9]:
#Creating the schema for the clickstream data json structure
clickjsonschema = StructType() \
.add("itemid", StringType()) \
.add("userid", StringType()) \
.add("device", StringType()) \
.add("sessionid", IntegerType()) \
.add("event_name", StringType()) \
.add("date", TimestampType())  


StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 11, Finished, Available)

## Define folder locations to store data
#### We store them in OneLake, under Files of our lakehouse

In [10]:
#Function to create required folders in mount point


def table_delta_dir(table="Customer"): 
  val = f"Files/clickstreamdata/{table}/" 
  return val

def table_chkpt_dir(table="Customer"): 
  val = f"Files/clickstreamdata/{table}/chkpnt/" 
  return val

def hist_dir(type="Hist"): 
  val = f"{path_to_storage}/clickstream-hist/{type}/Data" 
  return val


StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 12, Finished, Available)

## Event hubs for Apache Kafka using Spark streaming

#### Spark Structured Streaming has built-in support for reading and writing to Apache Kafka. The option startingOffsets controls how we consume the topic. latest: we consume only the new incoming events in the topic. earliest: we consume all the events present in the topic.

#### Create a function that will process the JSON input data and return a properly formatted Spark DataFrame that can be output to the Delta table. The checkpointPath tracks the Kafka data that’s already been processed, so you don’t need to manually track the data that’s already been ingested into the Delta table. Spark Structured Streaming makes these incremental updates quite easy.

#### We read the Kafka stream with the trigger to availableNow which allows for periodic processing. If you need Spark Structured Streaming to continuously process new data from the Kafka stream, use something like trigger(processingTime='10 seconds'). For continuous streaming, you need a cluster that’s always running. That is usually more expensive than periodically provisioning a new cluster and incrementally processing the new data. Continuous streaming is good for low-latency data processing pipelines. Incremental processing is good when the latency requirements can be higher.

#### We use Shared access signature (SAS) for authorizing access to consume events from Event Hubs. When using Apache Kafka protocol with your clients, you can set your configuration for authentication and encryption using the SASL mechanisms. Event Hubs for Kafka requires the TLS-encryption (as all data in transit with Event Hubs is TLS encrypted), which can be done specifying the SASL_SSL option in your configuration file. OAuth is another method.

#### Read more here: https://delta.io/blog/write-kafka-stream-to-delta-lake/
#### and https://learn.microsoft.com/en-us/azure/event-hubs/azure-event-hubs-kafka-overview
#### and https://learn.microsoft.com/en-us/azure/event-hubs/event-hubs-kafka-spark-tutorial

In [11]:
#Event Hubs for Kafka configuration details
BOOTSTRAP_SERVERS = "salabcommerce-eventhubs.servicebus.windows.net:9093"
EH_SASL = 'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://salabcommerce-eventhubs.servicebus.windows.net/;SharedAccessKeyName=youraccesskeyname;SharedAccessKey=xxxxxxxxxxxxxxxxxx";'
GROUP_ID = "$Default"

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 13, Finished, Available)

In [12]:
# Function to read data from EventHub and writing as delta format
def append_kafkadata_stream():    
    topic="clickstream-eventhub" # your Event Hub instance
    
  
    kafkaDF = (spark.readStream \
        .format("kafka") \
        .option("subscribe", topic) \
        .option("kafka.bootstrap.servers", BOOTSTRAP_SERVERS) \
        .option("kafka.sasl.mechanism", "PLAIN") \
        .option("kafka.security.protocol", "SASL_SSL") \
        .option("kafka.sasl.jaas.config", EH_SASL) \
        .option("kafka.request.timeout.ms", "60000") \
        .option("kafka.session.timeout.ms", "60000") \
        .option("kafka.group.id", GROUP_ID) \
        .option("failOnDataLoss", "false") \
        .option("startingOffsets", "latest") \
        .load().withColumn("source", lit(topic)))
   
    newkafkaDF=kafkaDF.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)","source").withColumn('clickstream', from_json(col('value'),schema=clickjsonschema))
    
    kafkajsonDF=newkafkaDF.select("key","value","source", "clickstream.*")
        
    query=kafkajsonDF.selectExpr(
                      "itemid"	   \
                      ,"userid"	\
                      ,"device" \
                      ,"sessionid" \
                      ,"event_name" \
                      ,"date" \
                      ,"source") \
                .writeStream.format("delta") \
                .trigger(availableNow=True) \
                .outputMode("append") \
                .option("checkpointLocation",table_chkpt_dir("retaildelta_bronze")) \
                .start(table_delta_dir("retaildelta_bronze")) 

    return query

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 14, Finished, Available)

In [49]:
# Reading data from EventHubs for Kafka

query_source1 = append_kafkadata_stream()

#run it one more time, just in case we missed to capture

query_source1 = append_kafkadata_stream()

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 51, Finished, Available)

## Read historical clickstream data from ADLS

In [14]:
# Function to read data from ADLS using readStream API (autoloader) and writing as delta format
def append_batch_source():
    topic ="clickstream-hist"

    histDF = (spark.readStream \
        .schema(clickjsonschema) \
        .format("csv") \
        .load(hist_dir("Hist")).withColumn("source", lit(topic))) # this needs to point to source FO ADLS data

    query=histDF.selectExpr(
                      "itemid"	   \
                      ,"userid"	\
                      ,"device" \
                      ,"sessionid" \
                      ,"event_name" \
                      ,"date" \
                      ,"source") \
                .writeStream.format("delta") \
                .trigger(availableNow=True) \
                .option("checkpointLocation",table_chkpt_dir("retaildelta_hist")) \
                .outputMode("append") \
                .start(table_delta_dir("retaildelta_hist")) 

    return query

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 16, Finished, Available)

In [15]:
# Reading data from Historical location ( in this example its from ADLS Gen-2 having historical data for clickstream)
# There may be cases where historical data can be added to this location from any other source where the schema is same for all the files. In such scenarios using readStream API on ADLS location will keep polling for new data and when available it will be ingested

query_source2 = append_batch_source()


StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 17, Finished, Available)

## Bronze tables

In [16]:
# Dropping all Delta tables if required
def DropDeltaTables(confirm=1):
  
  if(confirm ==1):
    spark.sql("DROP TABLE IF EXISTS RetailClickstream.RetailDelta_Bronze")
    spark.sql("DROP TABLE IF EXISTS RetailClickstream.RetailDelta_Silver")
    spark.sql("DROP TABLE IF EXISTS RetailClickstream.RetailDelta_Gold")
    spark.sql("DROP TABLE IF EXISTS RetailClickstream.RetailDelta_Hist")
    spark.sql("DROP TABLE IF EXISTS RetailClickstream.Customer_Silver")
    spark.sql("DROP TABLE IF EXISTS RetailClickstream.Product_Silver")

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 18, Finished, Available)

In [17]:
#Function which drops all delta tables. To avoid droping tables call the function with confirm=0
DropDeltaTables(confirm=1)

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 19, Finished, Available)

In [18]:
# Wait for 10 seconds before we create the delta tables else it might error out that delta location is not created
time.sleep(10)

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 20, Finished, Available)

In [19]:
'''
spark.sql (f"""
CREATE DATABASE IF NOT EXISTS {lakehouse}
"""
)
'''

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 21, Finished, Available)

'\nspark.sql (f"""\nCREATE DATABASE IF NOT EXISTS {lakehouse}\n"""\n)\n'

In [50]:
# Creating the delta table on delta location for Bronze data

# Load a file into a dataframe
df_bronze= spark.read.load(table_delta_dir("retaildelta_bronze"), format='delta', header=True)
# Save the dataframe as a delta table
df_bronze.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("retaildelta_bronze")

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 52, Finished, Available)

In [22]:
# Creating the delta table on delta location for Historical data

# Load a file into a dataframe
df_hist = spark.read.load(table_delta_dir("retaildelta_hist"), format='delta', header=True)
# Save the dataframe as a delta table
df_hist.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("retaildelta_hist")

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 24, Finished, Available)

In [23]:
%%sql
describe formatted RetailClickstream.RetailDelta_Bronze

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 25, Finished, Available)

<Spark SQL result set with 15 rows and 3 fields>

In [24]:
#Creating Temp View on Bronze DF
df_bronze.createOrReplaceTempView("vw_TempBronze")

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 26, Finished, Available)

In [51]:
%%sql
select count(*) from vw_TempBronze

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 53, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [1]:
%%sql
select * from vw_TempBronze limit 10

StatementMeta(, afbb98ef-a445-465c-b347-38ed7c93d2a8, 2, Finished, Cancelled)

In [27]:
%%sql
-- select count(*),hour(eventtime) as hour, day(eventtime) as day from vw_TempSilver group by hour(eventtime),day(eventtime)
select *, Year(date) as Year, month(date) as Month,day(date) as Day, hour(date) as Hour from vw_TempBronze limit 10

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 29, Finished, Available)

<Spark SQL result set with 0 rows and 11 fields>

In [28]:
#Joining both historical and Bronze Streaming Data
df_bronze_hist = df_bronze.union(df_hist)

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 30, Finished, Available)

In [29]:
df_bronze_hist.createOrReplaceTempView("vw_TempBronzeHistorical")

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 31, Finished, Available)

In [52]:
%%sql
select count(*) from vw_TempBronzeHistorical

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 54, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [31]:
%%sql
select * from vw_TempBronzeHistorical limit 10

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 33, Finished, Available)

<Spark SQL result set with 10 rows and 7 fields>

## Silver tables

In [32]:
  #run this code only afer you pushed some changes to source delta table, that populates the _change_data folder 
  #Since we want to use Change Data feed, we need to capture the version number, and use it as watermark for incremental
  
  df_cust = spark.sql (f"DESCRIBE HISTORY {lake_source_table1}")
  max_version = df_cust.agg({"version": "max"}).collect()[0][0]
     
  df_custSmall = df_custSmall.withColumn("_commit_version", lit(max_version))
  df_custSmall = df_custSmall.withColumn("_change_type", lit('insert'))
  df_custSmall = df_custSmall.withColumn("_commit_timestamp", lit(current_timestamp()))
  df_custSmall = df_custSmall.withColumn("_loadedtimestamp", lit(current_timestamp()))

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 34, Finished, Available)

In [33]:
# at this stage, you can write the df to a table
# this will be savedan external table as we dont load the df first !! if you load the df and then save, its a managed table

df_custSmall.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
.option("checkpointLocation",table_chkpt_dir("customer_silver")) \
.option("path", table_delta_dir("customer_silver")) \
.saveAsTable("customer_silver")

display(df_custSmall.limit(10))

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 35, Finished, Available)

SynapseWidget(Synapse.DataFrame, 20fa79b7-393d-46fe-970d-aebfd0d42c30)

In [34]:
df_productSmall.write.format("delta").mode("overwrite") \
.option("overwriteSchema", "true") \
.option("checkpointLocation",table_chkpt_dir("product_silver")) \
.option("path", table_delta_dir("product_silver")) \
.saveAsTable("product_silver")

display(df_productSmall.limit(10))

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 36, Finished, Available)

SynapseWidget(Synapse.DataFrame, 24c93220-2339-4ffe-a87c-4d5051eb762d)

In [57]:
# Create a silver delta table by joining Bronze view with Customers and Products views from Dynamics

df_silver= spark.sql("select s.*, c.CustomerGroup, c.Currency, p.ProductName, Year(date) as Year, month(date) as Month,day(date) as Day, \
                     hour(date) as Hour  \
                     from vw_TempBronzeHistorical s \
                     left join vw_Customers c on s.userid = c.CustomerId \
                     left join vw_Products p on s.itemid = p.ProductId") \
            .write.format("delta").option("MergeSchema","True").mode("Overwrite") \
            .option("checkpointLocation", table_chkpt_dir("retaildelta_silver")) \
            .option("path", table_delta_dir("retaildelta_silver")) \
            .saveAsTable("retaildelta_silver")

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 59, Finished, Available)

In [58]:

# Wait for 5 seconds before we create the delta tables else it might error out that delta location is not created
time.sleep(5)

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 60, Finished, Available)

In [59]:
%%sql
select count(*) from RetailClickstream.RetailDelta_Silver

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 61, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [38]:
%%sql
describe formatted RetailClickstream.RetailDelta_Silver

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 40, Finished, Available)

<Spark SQL result set with 22 rows and 3 fields>

In [39]:
%%sql
select * from RetailClickstream.RetailDelta_Silver limit 10

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 41, Finished, Available)

<Spark SQL result set with 10 rows and 14 fields>

## Gold tables

In [60]:
# create a Gold table with some aggregation for analytics purposes
df_gold=(spark.read.format("delta").option("latestFirst", "true").table("RetailClickstream.RetailDelta_Silver") \
                                 .groupBy(window('date',"1 hour"),"CustomerGroup","device","Month","Day","Hour").count()) \
                                .write.format("delta").mode("Overwrite").option("MergeSchema","True").option("checkpointLocation",table_chkpt_dir("retaildelta_gold")).option("path", table_delta_dir("retaildelta_gold")).saveAsTable("retaildelta_gold")

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 62, Finished, Available)

In [41]:
time.sleep(10)

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 43, Finished, Available)

In [54]:
df_gold =(spark.read.format("delta").table("retaildelta_gold"))
df_gold.createOrReplaceTempView("vw_GoldAggDetails")

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 56, Finished, Available)

In [61]:
%%sql
-- Viwing data from the Gold Delta Tables
select * from RetailClickstream.RetailDelta_Gold
ORDER BY Month DESC, Day Desc,count desc  limit 10

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 63, Finished, Available)

<Spark SQL result set with 10 rows and 7 fields>

In [62]:
%%sql
-- Viwing data from the Gold Delta Tables
select * from vw_GoldAggDetails limit 10

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 64, Finished, Available)

<Spark SQL result set with 10 rows and 7 fields>

In [63]:
# write Silver table data to Cosmos DB - define a config
"""
writeConfig = {
    "Endpoint": "https://salabcommerce-cosmosdb.documents.azure.com:443/",
    "Masterkey": "xxxxxxxxxxxxxxxxxxxxxxxx",
    "Database": "Retail",
    "Collection": "Clickstream",
    "Upsert": "true",
    "WritingBatchSize": "500"
   }
"""


StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 65, Finished, Available)

'\nwriteConfig = {\n    "Endpoint": "https://salabcommerce-cosmosdb.documents.azure.com:443/",\n    "Masterkey": "xxxxxxxxxxxxxxxxxxxxxxxx",\n    "Database": "Retail",\n    "Collection": "Clickstream",\n    "Upsert": "true",\n    "WritingBatchSize": "500"\n   }\n'

In [64]:
"""
changeFeed = (retailSilverDf
               .writeStream
               .format("com.microsoft.azure.cosmosdb.spark.streaming.CosmosDBSinkProvider")
               .outputMode("append")
               .options(**writeConfig)
               .option("checkpointLocation", "/mnt/commercedata/clickstreamdata/cosmos/chkpnt/")
               .start())
"""

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 66, Finished, Available)

'\nchangeFeed = (retailSilverDf\n               .writeStream\n               .format("com.microsoft.azure.cosmosdb.spark.streaming.CosmosDBSinkProvider")\n               .outputMode("append")\n               .options(**writeConfig)\n               .option("checkpointLocation", "/mnt/commercedata/clickstreamdata/cosmos/chkpnt/")\n               .start())\n'

## Analytics

In [65]:
%%sql

SELECT CustomerGroup, device, Month, Day, COUNT(*) as TotalDevices

FROM vw_GoldAggDetails

GROUP BY CustomerGroup, device , Month, Day

ORDER BY Month DESC, TotalDevices DESC

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 67, Finished, Available)

<Spark SQL result set with 9 rows and 5 fields>

In [66]:
%%sql

SELECT CustomerGroup, device, Month, COUNT(*) as TotalDevices

FROM vw_GoldAggDetails

GROUP BY CustomerGroup, device, Month

StatementMeta(, 31e0d6cd-ee3d-4e30-be74-8433efcc5665, 68, Finished, Available)

<Spark SQL result set with 9 rows and 4 fields>